$$
\newcommand{\mat}[1]{\boldsymbol {#1}}
\newcommand{\mattr}[1]{\boldsymbol {#1}^\top}
\newcommand{\matinv}[1]{\boldsymbol {#1}^{-1}}
\newcommand{\vec}[1]{\boldsymbol {#1}}
\newcommand{\vectr}[1]{\boldsymbol {#1}^\top}
\newcommand{\rvar}[1]{\mathrm {#1}}
\newcommand{\rvec}[1]{\boldsymbol{\mathrm{#1}}}
\newcommand{\diag}{\mathop{\mathrm {diag}}}
\newcommand{\set}[1]{\mathbb {#1}}
\newcommand{\cset}[1]{\mathcal{#1}}
\newcommand{\norm}[1]{\left\lVert#1\right\rVert}
\newcommand{\pderiv}[2]{\frac{\partial #1}{\partial #2}}
\newcommand{\bb}[1]{\boldsymbol{#1}}
\newcommand{\E}[2][]{\mathbb{E}_{#1}\left[#2\right]}
\newcommand{\ip}[3]{\left<#1,#2\right>_{#3}}
\newcommand{\given}[]{\,\middle\vert\,}
\newcommand{\DKL}[2]{\cset{D}_{\text{KL}}\left(#1\,\Vert\, #2\right)}
\newcommand{\grad}[]{\nabla}
\newcommand{\norm}[1]{\left\lVert#1\right\rVert}
$$

# Part 2: Summary Questions
<a id=part2></a>

This section contains summary questions about various topics from the course material.

You can add your answers in new cells below the questions.

**Notes**

- Clearly mark where your answer begins, e.g. write "**Answer:**" in the beginning of your cell.
- Provide a full explanation, even if the question doesn't explicitly state so. We will reduce points for partial explanations!
- This notebook should be runnable from start to end without any errors.

### CNNs

1. Explain the meaning of the term "receptive field" in the context of CNNs.

<font color='darkgreen'>
    
**Answer:** 
    
[*Receptive Field*](https://en.wikipedia.org/wiki/Receptive_field), of a biological neuron is the portion of the sensory space that can elicit neuronal responses, when stimulated.
In deep learning context, the receptive field is defined as the size of the region in the input that produces the feature.
As shown in [tutorial 4](https://nbviewer.jupyter.org/github/vistalab-technion/cs236781-tutorials/blob/master/t04/tutorial4-CNNs.ipynb?flush_cache=true) using the figure below, in CNNs each pixel in each layer may has information from the previous layer, depending on the architecture.
<figure>
<center>
<img src='https://theaisummer.com/assets/img/posts/receptive-field/receptive-field-in-convolutional-networks.png' width=300>
<figcaption>Receptive Field Illustration</figcaption></center>

    


2. Explain and elaborate about three different ways to control the rate at which the receptive field grows from layer to layer. Compare them to each other in terms of how they combine input features.

<font color='darkgreen'>
    
**Answer:** 

We can control the receptive field size using: 

* **Kernel Size:** the receptive fiels is proportional to the kernel size - for a kenel of size $k$, each pixel in the next layer has information of $k$ pixels from the previous layer.
* **Strides:**  the receptive fiels is proportional to the stride because the bigger the stride the less overlapping pixels we have. Note that, if the stride is bigger than the kernel size $k$, we will miss pixels.
* **Dilation:** See figure (D) here, compared to all other. 
For kernel of size $k$, a dilation of factor of $d$ (stride of $d$ between pixels in the source of the convolution) will increases the receptive field to $d \cdot (k - 1) + 1$ (because the kernel gets wider wider with holes between each adjacent kernel pixels). Therefore, for $d>1$ the region in the input that produces the feature gets bigger.
<!-- <figure>
<center>
    <img src='https://miro.medium.com/max/494/0*oX5IPr7TlVM2NpEU.gif' width=300>
    <figcaption>Dilation = 1</figcaption></center>
<figure>
<center>
    <img src='https://miro.medium.com/max/494/0*3cTXIemm0k3Sbask.gif' width=300>
        <figcaption>Dilation = 2</figcaption></center> -->

| (A) No paading, Stride = Dilation = 1 | (B) Padding = Stride = Dilation = 1 | (C) Stride = 2, Padding = Dilation = 1  | (D) No padding, Stride = 1, Dilation = 2 |
| --- | --- | --- | --- | 
|  <img src='https://raw.githubusercontent.com/vdumoulin/conv_arithmetic/master/gif/no_padding_no_strides.gif' width=200> | <img src='https://raw.githubusercontent.com/vdumoulin/conv_arithmetic/master/gif/same_padding_no_strides.gif' width=200>|  <img src='https://raw.githubusercontent.com/vdumoulin/conv_arithmetic/master/gif/padding_strides.gif' width=200>| <img src='https://raw.githubusercontent.com/vdumoulin/conv_arithmetic/master/gif/dilation.gif' width=200>| 
    

3. Imagine a CNN with three convolutional layers, defined as follows:

In [2]:
import torch
import torch.nn as nn

cnn = nn.Sequential(
    nn.Conv2d(in_channels=3, out_channels=4, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Conv2d(in_channels=4, out_channels=16, kernel_size=5, stride=2, padding=2),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Conv2d(in_channels=16, out_channels=32, kernel_size=7, dilation=2, padding=3),
    nn.ReLU(),
)

cnn(torch.rand(size=(1, 3, 1024, 1024), dtype=torch.float32)).shape

torch.Size([1, 32, 122, 122])

What is the size (spatial extent) of the receptive field of each "pixel" in the output tensor?

<font color='darkgreen'>
    
**Answer:** 

    
The receptive field for 1D input can be calculated using the next recursive formula: $RF_{l-1} = s_l \cdot (RF_l - 1) + d_l \cdot (k_l - 1) + 1$
where $RF_i$ is the receptive field in the layer $i$ and $RF_L=$ where $L$ is the last layer, $k_l$ is the kernel size, $s_i$ is the stride, $d_l$ is the dilation factor. Note that layers like ReLU doesn't affect the $RF$.
    
We assume that the padding isn't affect the $RF$ and they just used to correct edges effects, as we have here.
The explantion for this recursive formula is:
* the stride $s_l$ effect - is simply by miltiplygin the previous $RF_{l-1}$, and $s_l-1$ are skipped $\longrightarrow RF_{l-1}=s_l \cdot RF_l - (s_l - 1)$
* The kernel size $k_l$ and dilation $d_l$ - additive $k_l-1$ that cover margins (assumint that we using the correct padding, that is $\lfloor \frac{k_l-1}{2} \rfloor$). Additionaly, dilation $d_l \neq 1$ make the the convolution to use pixels that are not adjacence, and therefore it increses the additional term we mentioned before to $d_l \cdot (k_l - 1) + 1$

    
Summing this two terms, provided the recursive formula we write at the begging of the answer.
Here we have image as an input, so the anwer will be equal for both aces ($RF \times RF$).
    Using that formula, we can recursively calculate the original $RF$ of the architecture above, which denoted here as $RF_0$:

$
\text{ReLU layers:} \ RF_1 = RF_2 ; RF_7 = RF_8 ; RF_4 = RF_5 \\
\text{Final layer:} \ RF_8 = 1 \\
$
    
$
r_0 = 1 \cdot (RF_1 - 1) + 1 \cdot (3 - 1) + 1  = RF_1 + 2 = RF_2 + 2 =\\ 
= ( 2 \cdot (RF_3 - 1) + 1 \cdot (2 - 1) + 1 ) + 2 = \dots = 2 \cdot RF_3 + 2 =\\
= 2 \cdot ( 2 \cdot (RF_4 - 1) + 1 \cdot (5 - 1) + 1 ) + 2  = 4 \cdot RF_4 + 8 = 4 \cdot RF_5 + 8 = \\
= 4 \cdot ( 2 \cdot (RF_6 - 1) + 1 \cdot (2 - 1) + 1 ) + 8  = 8 \cdot RF_6 + 8 =\\
= 8 \cdot ( 1 \cdot (RF_7 - 1) + 2 \cdot (7 - 1) + 1 ) + 8  = 8 \cdot RF_7 + 104 = 8 \cdot RF_8 + 104  \\ \\
\Longrightarrow RF_0 = 8 + 104 = 112
$

In conclusion, for each axis we have $RF$ of 112, and we work on image and equal operation for both axes, so we have got $RF_0=112 \times 112$


4. You have trained a CNN, where each layer $l$ is represented by the mapping $\vec{y}_l=f_l(\vec{x};\vec{\theta}_l)$, and $f_l(\cdot;\vec{\theta}_l)$ is a convolutional layer (not including the activation function).

  After hearing that residual networks can be made much deeper, you decide to change each layer in your network you used the following residual mapping instead $\vec{y}_l=f_l(\vec{x};\vec{\theta}_l)+\vec{x}$, and re-train.

  However, to your surprise, by visualizing the learned filters $\vec{\theta}_l$ you observe that the original network and the residual network produce completely different filters. Explain the reason for this.

<font color='darkgreen'>

**Answer:** 

Basically, changing the network architecture is going to affect the weights values that optimizes the loss function.
For example, if we use the weights learned in the original network, in the residual network sense each layer is a different function (becuase we are adding $x$ to the function) obviously we will get the wrong output. Therefore, the weights which optimize the residual network has to be adjusted to the fact that the blocks are now different (residual).



### Dropout

1. Consider the following neural network:

In [ ]:
import torch.nn as nn

p1, p2 = 0.1, 0.2
nn.Sequential(
    nn.Conv2d(in_channels=3, out_channels=4, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.Dropout(p=p1),
    nn.Dropout(p=p2),
)

If we want to replace the two consecutive dropout layers with a single one defined as follows:
```python
nn.Dropout(p=q)
```
what would the value of `q` need to be? Write an expression for `q` in terms of `p1` and `p2`.

<font color='darkgreen'>
    
**Answer:** 

The `nn.Dropout(p)` layer randomly zeroes some elements of the input tensor with probability $p$ using samples from a Bernoulli distribution. Therfore after sequantial of `nn.Dropout(p=p1)` $\rightarrow$ `nn.Dropout(p=p2)` an element will zero if it zeroed in the first dropout layer $(p_1)$ <font color='red'>**OR**</font> in the second dropout layer $(p_2)$ (remember that they statistically independent).
Therefore, an equivalent layer `nn.Dropout(p=q)` will need to be with probability $q = Pr(dropout_1 \cup dropout_2) = p_1 + p_2 - p_1 \cdot p_2 = 0.1 + 0.2 - 0.1 \cdot 0.2 = 0.28$.
    
Simplier way to think of it calculate it as a complementary event, i.e., the probability of neuron to survive is a dropout layer is $1-p$ so what we need is the complementary even of both $(1-p_1)$<font color='red'> **AND** </font> $(1-p_2)$ and hence the equivalent is: $q = 1-Pr(\overline{dropout}_1 \cap \overline{dropout}_2) = 1-(1-p_1) \cdot (1-p_2) = 1-0.9 \cdot 0.8 = 0.28$

2. **True or false**: dropout must be placed only after the activation function.

<font color='darkgreen'>
    
**Answer:** 
    
False. Usually, the dopout is applied after the activation function, but it is not a must. For example, considering the ReLU activation function, it makes more sense to apply the dropout before because $ReLU(0) = 0$, so it more computationally efficient. However, it is not a necessary.
    
    

3. After applying dropout with a drop-probability of $p$, the activations are scaled by $1/(1-p)$. Prove that this scaling is required in order to maintain the value of each activation unchanged in expectation.

<font color='darkgreen'>
    
**Answer:** 
The dropout activation can be written as:
$$
    y_{dropout} = f(x)= 
\begin{cases}
    x,              & \text{with probability of} \ 1-p \\
    0,              & \text{with probability of} \ p
\end{cases}
$$
    
Let's calculate the expectation value $\mathbb{E}[y_{dropout}] = p\cdot 0 +(1-p) \cdot x = (1-p) \cdot x $
It easy to see that without the drop we have $\mathbb{E}[y_{without dropout}] = x$
and we got a scale factor of: $\frac{\mathbb{E}[y_{without dropout}]}{\mathbb{E}[y_{dropout}] } = \frac{1}{1-p} $

### Losses and Activation functions

1. You're training a an image classifier that, given an image, needs to classify it as either a dog (output 0) or a hotdog (output 1). Would you train this model with an L2 loss? if so, why? if not, demonstrate with a numerical example. What would you use instead?

<font color='darkgreen'>
    
**Answer:** 
We wouldn't choose L2 loss in this case, bacause it won't strongly penelize for wrong classification. We would choose a binary log cross entropy instead.
    
We will explain it with a numerical example: let assume that our classifier is completly wrong to classify a hotdog (1), that means it predict 0 (dog). So, the L2 loss will be $(1-0)^2 = 1$, where the binary cross entropy will be $- (0 \cdot log(1) + 1 \cdot log(0)) \longrightarrow \infty$.

In addition, using MSE means that we assume that the underlying data has been generated from a normal distribution. In Bayesian terms this means we assume a Gaussian prior. While in reality, a dataset that can be classified into two categories (i.e. binary) is not from a normal distribution but a Bernoulli distribution.
Besides, the MSE function is non-convex for binary classification. In simple terms, if a binary classification model is trained with MSE Cost function, it is not guaranteed to minimize the Cost function. This is because MSE function expects real-valued inputs in range($-\infty$, $\infty$), while binary classification models output probabilities in $[0,1]$ through the sigmoid/logistic function. 
    
<!-- <font color='red'>
===========
IMO No. https://towardsdatascience.com/why-using-mean-squared-error-mse-cost-function-for-binary-classification-is-a-bad-idea-933089e90df7 -->

2. After months of research into the origins of climate change, you observe the following result:

<center><img src="https://sparrowism.soc.srcf.net/home/piratesarecool4.gif" /></center>

You decide to train a cutting-edge deep neural network regression model, that will predict the global temperature based on the population of pirates in `N` locations around the globe.
You define your model as follows:

In [3]:
import torch.nn as nn

N = 42  # number of known global pirate hot spots
H = 128
mlpirate = nn.Sequential(
    nn.Linear(in_features=N, out_features=H),
    nn.Sigmoid(),
    *[
        nn.Linear(in_features=H, out_features=H), nn.Sigmoid(),
    ]*24,
    nn.Linear(in_features=H, out_features=1),
)

# print(mlpirate)

While training your model you notice that the loss reaches a plateau after only a few iterations.
It seems that your model is no longer training.
What is the most likely cause?

<font color='darkgreen'>
    
**Answer:** 
    
We assume that the most likely cause for this is the ["Vanishing Gradients" problem](https://en.wikipedia.org/wiki/Vanishing_gradient_problem), that happens when the gradients calculated during backpropagation are small and that can make the neuron to have no change in the values.
It make sense that this is the problem due to the fact that model is too deep and has a lot of sigmoid layers (25*(FC->Sigmoid)->FC). As we can see at the figure added below, the sigmoid gradient is $\in [0, \sim 0.25]$, so it make sense why the multipication of that 25 times will causes the gradients to vanish.
    
<figure>
<center>
    <img src='https://miro.medium.com/max/3000/1*6A3A_rt4YmumHusvTvVTxw.png' width=800>
        <figcaption>Sigmoid and derivative of sigmoid plots</figcaption></center> 


3. Referring to question 2 above: A friend suggests that if you replace the `sigmoid` activations with `tanh`, it will solve your problem. Is he correct? Explain why or why not.

<font color='darkgreen'>
    
**Answer:** 
No, he is wrong. Although the $tanh$ derivative is $\in [0,1]$ which is better, it still has gradients that less than $1$ value, and the fact that the network architecture is still too deep and the vanishing gradient problem will not be solved.
    
<figure>
<center>
    <img src='https://qph.fs.quoracdn.net/main-qimg-f1baf29cfdb09202b18e2179f4f41bfc' width=500>
        <figcaption>Sigmoid and derivative of sigmoid plots</figcaption></center> 



4. Regarding the ReLU activation, state whether the following sentences are **true or false** and explain:
  1. In a model using exclusively ReLU activations, there can be no vanishing gradients.
  1. The gradient of ReLU is linear with its input when the input is positive.
  1. ReLU can cause "dead" neurons, i.e. activations that remain at a constant value of zero.

<font color='darkgreen'>
    
**Answer:** 
    
A. True. The deriviative of ReLU is
    $
    \frac{d}{dx}ReLU(x) = \frac{d}{dx}max(0,x) = 
\begin{cases}
    1,              & x>0 \\
    0,              & x \leq 0  \ \text{($x=0$ is undefined but set to 0))}
\end{cases}
$ <br>
 Vanishing gradint can happens only when the derivative of the activation function is $\in (0, 1)$, because we can get kind of geomeric seriest which convergence to 0.
    
B. True. As mention in the previous section, for positive input the gradient is constantly $1$, so it is linear for positive input $x$.  
    
C. True. It is possible since negative inputs ReLU will outputs $0$ that can make the activation remain a value of $0$.

### Optimization

1. Explain the difference between: stochastic gradient descent (SGD), mini-batch SGD and regular gradient descent (GD).

<font color='darkgreen'>
    
**Answer:** 
<figure>
<center>
    <img src='https://suniljangirblog.files.wordpress.com/2018/12/descent.png' width=500>
    <figcaption>GD, SGD and mini-batch SGD visualization</figcaption></center>
<figure><\center>

<!-- https://suniljangirblog.wordpress.com/2018/12/13/variants-of-gradient-descent/
     -->
    
 
All the GD methods are iterative methods, which updates the parameters in a way that reduces the function value untill we reach a minimum.
* **GD:** In GD, we work with all the samples in out training in every run, the gradient of the loss function is computed exactly in each step.
* **SGD:** This is an approximation of GD. The gradient of the loss function is approximated by a gradient of a random single sample (that doesn't choosed before).
* **Mini-batch SGD:** the gradient of the loss function is approximated by a gradient of a batch of samples. In other words, this method uses principles from both GD (use batch) and SGD (use part of the data rather than all of it).


2. Regarding SGD and GD:
  1. Provide at least two reasons for why SGD is used more often in practice compared to GD.
  2. In what cases can GD not be used at all?

<font color='darkgreen'>
    
**Answer:** 
    
A. SGD is more often used in practice becuase:

1. In order to use GD we need to use **all** the training set for each update of weights. if the training set is large, it can take a long runtime to reach the mininum by using GD.
    
2. Sometimes the training set is too large to even fit into the memory (all at once). In this case we can not use GD.
    
B. In the case where the training data is too big to fit in memory (all at once) we can not use GD.

3. You have trained a deep resnet to obtain SoTA results on ImageNet.
While training using mini-batch SGD with a batch size of $B$, you noticed that your model converged to a loss value of $l_0$ within $n$ iterations (batches across all epochs) on average.
Thanks to your amazing results, you secure funding for a new high-powered server with GPUs containing twice the amount of RAM.
You're now considering to increase the mini-batch size from $B$ to $2B$.
Would you expect the number of of iterations required to converge to $l_0$ to decrease or increase when using the new batch size? explain in detail.

<font color='darkgreen'>
    
**Answer:** 
    
We expect that the number of iterations it takes to converge to $l_0$ will decrease - we will converge faster.
Basically, as a rule of thumb, larger batch size create more robust and accurate model.
It can be seen from the image in question #1 in this section, that increasing the batch size will decrease the number of iterations to convergence.
Since the ideal proccess is GD - which uses the whole data it converges the fastest, increasing the batch size will get us closer to the ideal proccess, and therefore we will converge faster.
    

4. For each of the following statements, state whether they're **true or false** and explain why.
  1. When training a neural network with SGD, every epoch we perform an optimization step for each sample in our dataset.
  1. Gradients obtained with SGD have less variance and lead to quicker convergence compared to GD.
  1. SGD is less likely to get stuck in local minima, compared to GD.
  1. Training  with SGD requires more memory than with GD.
  1. Assuming appropriate learning rates, SGD is guaranteed to converge to a local minimum, while GD is guaranteed to converge to the global minimum.
  1. Given a loss surface with a narrow ravine (high curvature in one direction): SGD with momentum will converge more quickly than Newton's method which doesn't have momentum.


<font color='darkgreen'>
    
**Answer:** 
        
A. True - SGD using randomly one sample and avoid repeating the same sample, thus it iterate over the whole dataset in each epoch. Sometimes, this process is described by shuffling the whole database every epoch and using samples one after the other by the predefined order.
    
<!--     ####NOT SURE#### False - In SGD every iteration we choose stochastically one sample, and update the parameters taking into consideration only this one sample and it is chosen stochastically.
So after one epoch we might use the same sample a couple of times, or not use a sample at all.
     -->
B. False - Exactly the opposite. Since it uses only one example from the data, the update process is vety noisy with high variance. In the expectation SGD converges to GD, but when acctually performing SGD we might take steps "the wrong way", but on average we are progressing towards the minumum. so the variance is greater in SGD.
    
C. True - If we are currently in a local minima, GD is stuck, because when using GD we compute the exact gradient of the function, which is 0. But, SGD might get itself out of the local minima, since it uses only some of the sample and not the whole data set. So the gradient is not exatly 0, so if we keep iterating we can escape this local minima. In other words, the noise we described in section B here, might help the SGD in this case to get out of the local minima.
    
D. False - Using GD we need all our dataset for each iteration. Using SGD we just use a minibatch (or maybe even one sample) each iteration.
    
E. False - Both methods guarantees to converge only to a local minima. If the funciton is not convex the minimal value we converge to some minima depends on the initialization of the parameters - and it may be a local minima. 
    
F True - Using momentum can help in this case because the momentum accumulates the gradients from previous iterations (from the beginning until current step with decay factor), if the gradient in current iteration has the same direction as the momentum, then the weight-update in the same direction will have larger size step and thus in our case it will coverge faster. See visuallization below.
    
<figure>
    <center>
        <img src='https://media.springernature.com/lw785/springer-static/image/chp%3A10.1007%2F978-1-4842-4470-8_33/MediaObjects/463852_1_En_33_Fig1_HTML.jpg' width=400>
        <figcaption>The effect of using momentum in SGD method</figcaption>
    </center>
<figure>
    
    
    
<!-- *in order to use newton's method, we need to caculate the hessian matrix which has a dimention of $n*n$ where n is the number of parameters.
   
*in order to use SGD with momentum, we only need to calculate the gradient of the current step, and accumulate gradients from the past steps.
Furthermore, the fact that we use the momentum, will enlarge the gradient in the direction of the ravine.
These are the reasons that SGD with momentum will converge faster. -->



5. In tutorial 5 we saw an example of bi-level optimization in the context of deep learning, by embedding an optimization problem as a layer in the network.
  **True or false**: In order to train such a network, the inner optimization problem must be solved with a descent based method (such as SGD, LBFGS, etc).
  Provide a mathematical justification for your answer.

<font color='darkgreen'>
    
**Answer:** 
    
False - 
    As shown in the tutorial, we want to find 
    $\tilde{y}$ such that: 
    $$\tilde{y}=argmin_y f(y;z)$$
    
We also know the following:
    
* If the gradient of $f$ is $0$ for a given $y$, the perturbing $z \rightarrow z+dz$ will "move" the minimum to different $y+dy$.
    
* Therefore, we can write using Taylor: 
$$\nabla_y f(y+dy,z+dz) \approx \nabla_y f(y,z)+\nabla^2_{yy} f(y,z)dy+\nabla^2 _{yz} f(y,z)dz=0 $$
    
* because: 
$$\nabla_y f(y,z)=0$$
    we obtain: 
    $$\nabla^2_{yy} f(y,z)dy=-\nabla^2 _{yz} f(y,z)dz$$
and then we can write:
    $$dy =-[\nabla^2_{yy}f(y,z)]^{-1}\nabla^2 _{yz}f(y,z)dz $$
So we got the term we need in order to calculate the gradient without a decent based method.
    

6. You have trained a neural network, where each layer $l$ is represented by the mapping $\vec{y}_l=f_l(\vec{x};\vec{\theta}_l)$ for some arbitrary parametrized functions $f_l(\cdot;\vec{\theta}_l)$.
  Unfortunately while trying to break the record for the world's deepest network, you discover that you are unable to train your network with more than $L$ layers.
  1. Explain the concepts of "vanishing gradients", and "exploding gradients".
  2. How can each of these problems be caused by increased depth?
  3. Provide a numerical example demonstrating each.
  4. Assuming your problem is either of these, how can you tell which of them it is without looking at the gradient tensor(s)?

<font color='darkgreen'>
    
**Answer:** 
    
A. vanishing gradients is a case where the gradient are very close to 0. 
    exploading gradients is a case where the gradient are getting bigger and bigger and approach infinity. 
    In either way it is hard to learn.
    
B. In the back propagation algorithms we multiplay the gradients. the number of gradient which are being multiplyed is proportion to the number of layers.
    if the gradients are less then one and the number of layers is big. then they will vanish (it is a geometric series with q less then 1) - vanishing gradients.
    if the gradients are greater then one and the number of layers is big. then they will explode (it is a geometric series with q greater then 1) - exploading gradients.
    
C. Lets consider a case where we have K hidden layers.
    Therefore the update for the first layer is: $\pderiv{L}{W_1} = \pderiv{L}{h^k} \cdot \pderiv{h^k}{h^{k-1}} \cdot \pderiv{h^(k-1)}{h^{k-2}} ... \pderiv{h^(1)}{W_1}$
    We have K+1 terms here if each term here is less than 0.1 and $q<1$, then we get: $\pderiv{L}{W_1} < 0.1^{K+1}$ which where K is big it converges to 0 - vanishing gradients.
    if each term here is greater than 1.1 and $q<1$, then we get: $\pderiv{L}{W_1} < 1.1^{K+1}$ which where K is big it converges to infinity - exploading gradients.
    
D. We can identify the case by looking at the lose funciton:
    if the loss is stationary we have vanishing gradiants.
    if the loss is changing rapidly (in each iteration) we have exploading gradients.
    
    
    
    

### Backpropagation

1. You wish to train the following 2-layer MLP for a binary classification task:
  $$
  \hat{y} =\mat{W}_2~ \varphi(\mat{W}_1 \vec{x}+ \vec{b}_1) + \vec{b}_2
  $$
  Your wish to minimize the in-sample loss function is defined as
  $$
  L_{\mathcal{S}} = \frac{1}{N}\sum_{i=1}^{N}\ell(y,\hat{y}) + \frac{\lambda}{2}\left(\norm{\mat{W}_1}_F^2 + \norm{\mat{W}_2}_F^2 \right)
  $$
  Where the pointwise loss is binary cross-entropy:
  $$
  \ell(y, \hat{y}) =  - y \log(\hat{y}) - (1-y) \log(1-\hat{y})
  $$
  
  Write an analytic expression for the derivative of the final loss $L_{\mathcal{S}}$ w.r.t. each of the following tensors: $\mat{W}_1$, $\mat{W}_2$, $\mat{b}_1$, $\mat{b}_2$, $\mat{x}$.

<font color='darkgreen'>
    
**Answer:** 
* w.r.t. $W_1$:
$$\frac{\partial L_s}{\partial W_1} =\frac{1}{N} \sum \frac{\partial l}{\partial W_1} + \lambda ||W_1||$$
Where the following stands:
$$\frac{\partial l}{\partial W_1} = -y \cdot \frac{\partial log(\hat{y})}{\partial W_1} - (1-y) \frac{\partial log(1-\hat{y})}{\partial W_1}$$
$$\frac{\partial log(\hat{y})}{\partial W_1} = \frac{1}{\hat{y}} \frac{\partial \hat{y}}{\partial W_1}$$
$$\frac{\partial log(1-\hat{y})}{\partial W_1} = -\frac{1}{1-\hat{y}} \frac{\partial \hat{y}}{\partial W_1}$$
$$\frac{\partial \hat{y}}{\partial W_1} = W_2 \phi'(W_1x+b_1)x^t $$
* w.r.t. $W_2$:
$$\frac{\partial L_s}{\partial W_2} =\frac{1}{N} \sum \frac{\partial l}{\partial W_2} + \lambda ||W_2||$$
Where the following stands:
$$\frac{\partial l}{\partial W_2} = -y \cdot \frac{\partial log(\hat{y})}{\partial W_2} - (1-y) \frac{\partial log(1-\hat{y})}{\partial W_2}$$
$$\frac{\partial log(\hat{y})}{\partial W_2} = \frac{1}{\hat{y}} \frac{\partial \hat{y}}{\partial W_2}$$
$$\frac{\partial log(1-\hat{y})}{\partial W_2} = -\frac{1}{1-\hat{y}} \frac{\partial \hat{y}}{\partial W_2}$$
$$\frac{\partial \hat{y}}{\partial W_2} = \phi(W_1x+b_1) $$
    
* w.r.t. $b_1$:
$$\frac{\partial L_s}{\partial b_1} =\frac{1}{N} \sum \frac{\partial l}{\partial b_1}$$
Where the following stands:
$$\frac{\partial l}{\partial b_1} = -y \cdot \frac{\partial log(\hat{y})}{\partial b_1} - (1-y) \frac{\partial log(1-\hat{y})}{\partial b_1}$$
$$\frac{\partial log(\hat{y})}{\partial b_1} = \frac{1}{\hat{y}} \frac{\partial \hat{y}}{\partial b_1}$$
$$\frac{\partial log(1-\hat{y})}{\partial b_1} = -\frac{1}{1-\hat{y}} \frac{\partial \hat{y}}{\partial b_1}$$
$$\frac{\partial \hat{y}}{\partial b_1} = W_2 \phi'(W_1x+b_1)  $$

* w.r.t. $b_2$: 
$$\frac{\partial L_s}{\partial b_2} =\frac{1}{N} \sum \frac{\partial l}{\partial b_2}$$
Where the following stands:
$$\frac{\partial l}{\partial b_2} = -y \cdot \frac{\partial log(\hat{y})}{\partial b_2} - (1-y) \frac{\partial log(1-\hat{y})}{\partial b_2}$$
$$\frac{\partial log(\hat{y})}{\partial b_2} = \frac{1}{\hat{y}} \frac{\partial \hat{y}}{\partial b_2}$$
$$\frac{\partial log(1-\hat{y})}{\partial b_2} = -\frac{1}{1-\hat{y}} \frac{\partial \hat{y}}{\partial b_2}$$
$$\frac{\partial \hat{y}}{\partial b_1} =\vec{1}$$

2. Derivative a function $f(\vec{x})$ at a point $\vec{x}_0$ is
  $$
  f'(\vec{x}_0)=\lim_{\Delta\vec{x}\to 0} \frac{f(\vec{x}_0+\Delta\vec{x})-f(\vec{x}_0)}{\Delta\vec{x}}
  $$
  
  1. Explain how this formula can be used in order to compute gradients of neural network parameters numerically, without automatic differentiation (AD).
  
  2. What are the drawbacks of this approach? List at least two drawbacks compared to AD.

<font color='darkgreen'>
    
**Answer:** 
    
<!-- <font color='red'>
 -->
A. We can take a small number $\Delta\vec{W}$ and calculate the function value in $\vec{W}_0$ and $\vec{W}_0+\Delta\vec{W}$ (where $\vec{W}$ stands for a paramerter of NN) and compute the value of the function above. That would be the numerical derivative of the NN with respect to $\vec{W}$.
    We can do this calculation for each parameter $\vec{W}$ in the NN and get the gradient vector numerically without automatic differentition.

    
    
B. Two drawback are:
    
1. Accuracy, this  $\Delta\vec{W}$ we take will be finite, and not actually approaching to 0. Therefore this method will have an approximation error.
2. This method is an approximation and another drawback might be that it can produces a graident where the function is not differentiable.

3. Given the following code snippet:
  1. Write a short snippet that implements that calculates gradient of `loss` w.r.t. `W` and `b` using the approach of numerical gradients from the previous question.
  2. Calculate the same derivatives with autograd.
  3. Show, by calling `torch.allclose()` that your numerical gradient is close to autograd's gradient.

In [1]:
import torch

N, d = 100, 5
dtype = torch.float64

X = torch.rand(N, d, dtype=dtype)
W, b = torch.rand(d, d, requires_grad=True, dtype=dtype), torch.rand(d, requires_grad=True, dtype=dtype)

def foo(W, b):
    return torch.mean(X @ W + b)

loss = foo(W, b)
print(f"{loss=}")

# TODO: Calculate gradients numerically for W and b
epsilon = 0.00001
W_1 = torch.clone(W)
b_1 = torch.clone(b)


grad_W = torch.zeros(d, d, dtype=dtype)
grad_b = torch.zeros(d, dtype=dtype)
for i in range(d):
    for j in range(d):
        W_1[i][j] = W[i][j] + epsilon
        grad_W[i][j] = (foo(W_1,b) - (foo(W,b)))/epsilon
        W_1[i][j] = W[i][j]

for i in range(d):
        b_1[i] = b[i] + epsilon
        grad_b[i] = (foo(W,b_1) - (foo(W,b)))/epsilon
        b_1[i] = b[i]

# TODO: Compare with autograd using torch.allclose()
loss.backward()
autograd_W = W.grad
autograd_b = b.grad

assert torch.allclose(grad_W, autograd_W)
assert torch.allclose(grad_b, autograd_b)

loss=tensor(1.5318, dtype=torch.float64, grad_fn=<MeanBackward0>)


### Sequence models

1. Regarding word embeddings:
  1. Explain this term and why it's used in the context of a language model.
  1. Can a language model like the sentiment analysis example from the tutorials be trained without an embedding? If yes, what would be the consequence for the trained model? if no, why not?

<font color='darkgreen'>
    
**Answer:** 

A. word embeddings, is the representation of the word in a meaningful way (a vector), that encodes the meaning of the words in such a way that it can identify similarities between 2 words, by their representation in the vector space.

B. It can train, but the results would be poor, since it would not be able to identify the context of the words, moreover, it would not be able to identify the similaritier\unsimilarities between differrent words.

2. Considering the following snippet, explain:
  1. What does `Y` contain? why this output shape?
  2. How you would implement `nn.Embedding` yourself using only torch tensors. 

In [3]:
import torch.nn as nn

X = torch.randint(low=0, high=42, size=(5, 6, 7, 8))
embedding = nn.Embedding(num_embeddings=42, embedding_dim=42000)
Y = embedding(X)
print(f"{Y.shape=}")

Y.shape=torch.Size([5, 6, 7, 8, 42000])


<font color='darkgreen'>
    
**Answer:** 
    
A.num_embeddings is the number of words in the vocab.
    embedding_dim is the dimention of words vector we are using.
    So Y will be equal to the words imbedding which is given by the indices of X.
    The shape of Y is the same as X multiplied by the embedding dimention, because it replaces each index with a words. so 5x6x7x8 goes to (5x6x7x8)x42000.
    
B. As i said earlier num_embeddings is the number of words in the vocab.
    embedding_dim is the dimention of words vector we are using.
    So I would create a NN with an imput dimention of 42, and output dimention of 42000. every index (a number between 0-42) will be mapped to a vector which is 42000 dimentional.
    the mapping will be learned by training.
    

3. Regarding truncated backpropagation through time (TBPTT) with a sequence length of S: State whether the following sentences are **true or false**, and explain.
  1. TBPTT uses a modified version of the backpropagation algorithm.
  2. To implement TBPTT we only need to limit the length of the sequence provided to the model to length S.
  3. TBPTT allows the model to learn relations between input that are at most S timesteps apart.

<font color='red'>
    
**Answer:** 
    
A. True - The only modification to backpropagation, will be to accumulate gradients on in length S. since thats the only " short term memory" we want.
    
B. False - we don't have to limit the sequence length, we can just limit the number of timesteps of the backpropagation algorithm to length S.
    
C. True - since we limit the gradient accumulation to length S, we would only have "memory" of at most S timesteps.

### Attention

1. In tutorial 7 (part 2) we learned how to use attention to perform alignment between a source and target sequence in machine translation.
  1. Explain qualitatively what the addition of the attention mechanism between the encoder and decoder does to the hidden states that the encoder and decoder each learn to generate (for their language). How are these hidden states different from the model without attention
  
  2. After learning that self-attention is gaining popularity thanks to the shiny new transformer models, you decide to change the model from the tutorial: instead of the queries being equal to the decoder hidden states, you use self-attention, so that the keys, queries and values are all equal to the encoder's hidden states (with learned projections). What influence do you expect this will have on the learned hidden states?


<font color='darkgreen'>
    
**Answer:** 
    
<font color='red'>
    
A. Without attention the decoder recieves the hidden state from the encoder as is.
    By adding attention, the decoder will focus on differente parts of the sequence.
    The attention (soft attention mechanism) is a weighted average of the encoder ouputs, that match the current decoder state.

    
    
B. In this case the hidden states of the decoder are not used - so they won't be learned.
   What is going to happen, is that the hidden states of the encoder will change - they will be similar to the meaning of the sentence. 
   


### Unsupervised learning

1. As we have seen, a variational autoencoder's loss is comprised of a reconstruction term and  a KL-divergence term. While training your VAE, you accidentally forgot to include the KL-divergence term.
What would be the qualitative effect of this on:

  1. Images reconstructed by the model during training ($x\to z \to x'$)?
  1. Images generated by the model ($z \to x'$)?

<font color='darkgreen'>
    
**Answer:** 
    
A. in order to reconstract the image, we only need to use both the encoder and decoder, therefore we need the KL-divergence term (which is responsible for encoding) and the reconstruction term (which is responsibale for decoding) optimized.
    
B. Here the fact that we forgot to include the KL-divergence term is not going to damage us,because optimizing the KL-divergence term is responsible for the encoder, and not the decoder which generates samples.

2. Regarding VAEs, state whether each of the following statements is **true or false**, and explain:
  1. The latent-space distribution generated by the model for a specific input image is $\mathcal{N}(\vec{0},\vec{I})$.
  2. Every time we feed an image to the encoder, then decode the result, we'll get the same reconstruction.
  3. Since the real VAE loss term is intractable, what we actually minimize instead is it's upper bound, in the hope that the bound is tight.

<font color='darkgreen'>

**Answer:** 

A. TRUE - thats the way we train the model.

B. FALSE - There is a sampling envolved in this proccess therefore it is not deterministic.

C. TRUE - we optimize the KL-divergence term by optimizing the ELBO - evidence lower bound.

2. Regarding GANs, state whether each of the following statements is **true or false**, and explain:
  1. Ideally, we want the generator's loss to be low, and the discriminator's loss to be high so that it's fooled well by the generator.
  2. It's crucial to backpropagate into the generator when training the discriminator.
  3. To generate a new image, we can sample a latent-space vector from $\mathcal{N}(\vec{0},\vec{I})$.
  4. It can be beneficial for training the generator if the discriminator is trained for a few epochs first, so that it's output isn't arbitrary.
  5. If the generator is generating plausible images and the discriminator reaches a stable state where it has 50% accuracy (for both image types), training the generator more will further improve the generated images.

<font color='darkgreen'>
    
**Answer:** 
    
A. False. We want both of the disctriminator and the generator to work good and have a low losses. due to the fact that they train one against the other, we want them to have "fair competition". In other words, it will be hard to make a good discriminator (low loss) to have wrong decision and hence the generator will need to do a better job.
    
B. False. they are being train seperatly, so when we train the discriminator, we use as input the generator's output and we do not train it.
    
C. True. The generator maps a latent-space variable $u\sim \mathcal{N}(0, I)$ to instance-space varibale $x$, which is an image. Therefore a  parametric evidence distribution $(p_\gamma(X))$ is generated and we try to make it as closer as possible to the real evidence distribution $p(X)$.
    
D. False. At the beginning of the process, both the discriminator and the generator has bad performence, and hence it won't help us to have a better discriminator at the first steps. In addition, to create a good trainning for the discriminator, we need also good "fake" images, which need to be created for the untrained generator, thus it doesn't help us to train the discriminator for few epoch before.
###THIS IS THE REF,WHAT DO U WANNA DO?### True, we would want a decent discriminator in order to achieve better training of the generator.
    
E. False. If the discriminator has 50% accuracy, the generator will not be able to learn how to generate, because effectivly the discriminator is "helping" the generator to generate, so there is no use in training only the generator in this case.
    
    

### Graph Neural Networks

1. You have implemented a graph convolutional layer based on the following formula, for a graph with $N$ nodes:
$$
\mat{Y}=\varphi\left( \sum_{k=1}^{q} \mat{\Delta}^k \mat{X} \mat{\alpha}_k + \vec{b} \right).
$$
  1. Assuming $\mat{X}$ is the input feature matrix of shape $(N, M)$: what does $\mat{Y}$ contain in it's rows?
  1. Unfortunately, due to a bug in your calculation of the Laplacian matrix, you accidentally zeroed the row and column $i=j=5$ (assume more than 5 nodes in the graph).
What would be the effect of this bug on the output of your layer, $\mat{Y}$?

<font color='darkgreen'>
    
**Answer:** 

1. A. As we saw in tutorial 10:
$$y^{j} = \varphi\left( \sum_{i=1}^{m} \sum_{k=1}^{q} \alpha_{k}^{ij}\Delta^{k}x^{i}+b^{j} \right)$$

B. By looking at the formula above, if $x^{5} = 0$ we get:
$$y^{5} = \varphi\left( b^{5} \right)$$
And the rest is according to the formula.

2. We have discussed the notion of a Receptive Field in the context of a CNN. How would you define a similar concept in the context of a GCN (i.e. a model comprised of multiple graph convolutional layers)?

<font color='darkgreen'>
    
**Answer:**


2. In GCN, the most intuative way to define the concept of receptive field (for a target node), is their whole L-hop neighbors, where L is the number of layers.
    
    
